In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import gc
import os

In [2]:
user_id = 'ninaman'
#user_id = 'nightofwings'

domain = 'https://www.pttweb.cc'
url = '/user/{}?t=message'.format(user_id)
output_path = '/Users/zoe/Desktop/ptt_crawler/github_data/'

usr_result = []
page_lst = []
replace_ele = ['                ', '\n', '    ']

In [3]:
def reply_parser(reply_soup):
    reply_lst = []
    for push in reply_soup.select('.e7-top'):
        push_content = push.select('.yellow--text')[-1].text.strip(':')
        #if cross year, 2019 > 2020, check month diff
        push_datetime_pre = push.select('.ml-3')[0].text
        push_floor = push.select('.e7-floor')[0].text
        push_tag = push.select('.mr-2')[-1].text
        reply_lst.append([push_content, push_datetime_pre, push_floor, push_tag])
    return pd.DataFrame(reply_lst, columns=['push_content', 'push_datetime_pre', 'push_floor', 'push_tag'])

In [4]:
def page_parser(page_soup):
    rs_lst = []
    for x in page_soup.select('.thread-item'):
        reply_df = reply_parser(x)
        #article info
        reply_df['article_date'] = x.select('.thread-posttime')[0].text
        reply_df['article_lnk'] = x.a['href']
        reply_df['article_title'] = x.a.text
        ath_text = x.select('.thread-list-author')[0].text
        for _e in replace_ele:
            ath_text = ath_text.replace(_e, '')
        reply_df['author'] = ath_text.split('- 發表於')[0][3:]
        #reply summary
        post_ele = x.select('.thread-popularity')[0].text.split(',')
        reply_df['art_all'] = post_ele[0].strip(' 留言')
        reply_df['art_count'] =  post_ele[1].strip(' 推噓總分: ')

        rs_lst.append(reply_df)
    return rs_lst

In [5]:
pg_cnt = 0

while True:
    rs = requests.get(domain + url)
    soup = BeautifulSoup(rs.content)
    #parser
    cur_page = [x for x in soup.select('.e7-active')][0].text
    next_url = soup.find_all('a', text='下一頁')
    page_result = page_parser(soup)
    usr_result.extend(page_result)    
    #page record
    page_lst.append([cur_page, url])
    
    if pg_cnt % 50 == 0:
        print('current page is {}, url is {}'.format(pg_cnt, url))
        print(time.ctime())
    
    if not next_url:
        break
    
    url = next_url[0]['href']
    pg_cnt = pg_cnt + 1
    gc.collect()

current page is 1, url is /user/ninaman?t=message
Sun Jan 12 10:05:32 2020
current page is 51, url is /user/ninaman?t=message&page=50
Sun Jan 12 10:06:52 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1554096240000000000&k2=544380531
Sun Jan 12 10:08:11 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1552195920000000000&k2=537957414
Sun Jan 12 10:09:31 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1551188940000000000&k2=534648007
Sun Jan 12 10:10:51 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1550196000000000000&k2=531232328
Sun Jan 12 10:12:10 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1549209480000000000&k2=529850585
Sun Jan 12 10:13:30 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1548311100000000000&k2=526219188
Sun Jan 12 10:14:48 2020
current page is [ Δ0 ], url is /user/ninaman?t=message&k1=1547362320000000000&k2=523231339
Sun Jan 12 10:16:09 2020
current page is [ Δ0 ], url i

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

# output result

In [ ]:
rs_df = pd.concat(usr_result)
rs_df.replace(',', '，', regex=True, inplace=True)
rs_df.reset_index(drop=True, inplace=True)

rs_df_split = pd.concat([rs_df, rs_df.article_lnk.str.split('/', expand=True)[[2, 3]]], axis=1)
rs_df_split.rename({2:'board', 3:'article_id'}, axis=1, inplace=True)

In [27]:
rs_df_split.to_csv(output_path + '{}_reply.csv'.format(user_id), index=False)

# checking result

In [7]:
rs_df_split.groupby('push_tag').agg({'board':'count'})

,board
push_tag,
→,70
噓,6
推,97


In [9]:
#next & candidate union > over 200 pages have some isuues...
#narrow down to boards...
#rs_df.shape

(12100, 12)

In [9]:
board
push_tag	
→	12253
噓	1510
推	14410

留言數量: 28172
送出的『推』: 14412
送出的『→』: 12250
送出的『噓』: 1510
    
push_tag	
→	8279
噓	1699
推	12212
rs_df.groupby('push_tag').agg({'author':'count'})

,author
push_tag,
→,8279
噓,1699
推,12212
